# Class Weight Compution

In [1]:
import math
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
    
%matplotlib inline
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True) # don't use scientific notation

In [2]:
export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/'
# export_ds = '/media/scratch/berlukas/nuscenes'

# sem_clouds_filename = f"{export_ds}/sem_bkp/sem_classes_gt1.npy"
sem_clouds_filename = f"{export_ds}/sem_clouds2.npy"

print(f"Loading sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename).astype(int)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading sem clouds from /media/berlukas/Data2/datasets/nuscenes/processed//sem_clouds2.npy
Shape of sem clouds is (11452, 3, 200, 200)


In [2]:
export_ds = '/media/scratch/berlukas/nuscenes/training'
samples = os.listdir(export_ds)
sem_cloud_features = None
for sample in samples:
    sem_clouds_filename = f"{export_ds}/{sample}"    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename).astype(int)
    else:
        features = np.load(sem_clouds_filename).astype(int)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_poss_00_04_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_kitti_00_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_kitti_02_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_nuscenes_01_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_kitti_05_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_a2d2_20180925_135056_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_waymo_00_01_07_to_11_bw50.npy
Shape of sem clouds is (21184, 2, 100, 100)


In [3]:
n_classes = 6
class_counts = [0] * n_classes
for cl in range(0, n_classes): 
    count = np.count_nonzero(sem_cloud_features[:,1,:,:] == cl)
    print(f'Computing occurrences for {cl}: {count}')
    class_counts[cl] = count

Computing occurrences for 0: 190259862
Computing occurrences for 1: 1911822
Computing occurrences for 2: 101024
Computing occurrences for 3: 11303331
Computing occurrences for 4: 3686519
Computing occurrences for 5: 4577442


In [4]:
print(f'Counts are \n{class_counts}')
total_counts = np.sum(class_counts)
rates = class_counts / total_counts
print(f'Should sum to one: {np.sum(rates) == 1.0}')
print(f'\nRates are\n====================\n')
print(f'Vehicle: {rates[1]}')
print(f'Living: {rates[2]}')
print(f'Ground: {rates[3]}')
print(f'Manmade: {rates[4]}')
print(f'Vegetation: {rates[5]}')



inv_rates = 1-rates
# inv_rates[0] = 0
# inv_rates[8] = 0
# weights = np.round_((inv_rates) / np.sum(inv_rates), 20)

rates = rates[0:]
# weights = np.power(np.amax(rates) / rates, 1 / 3.0)
weights = 1 / rates

print(f'\nWeights are\n====================\n')
print(f'Ignore: {weights[0]}')
print(f'Vehicle: {weights[1]}')
print(f'Living: {weights[2]}')
print(f'Ground: {weights[3]}')
print(f'Manmade: {weights[4]}')
print(f'Vegetation: {weights[5]}')

print(f'\n ================= \n')
print(f'Weights sum to one: {(np.sum(weights) - 1.0) < 1e-3} ({np.sum(weights)})')

print(f'weights: {weights}')

Counts are 
[190259862, 1911822, 101024, 11303331, 3686519, 4577442]
Should sum to one: False

Rates are

Vehicle: 0.009024839501510574
Living: 0.0004768882175226586
Ground: 0.05335786914652568
Manmade: 0.017402374433534745
Vegetation: 0.021608015483383685

Weights are

Ignore: 1.1134245435329917
Vehicle: 110.80529463517001
Living: 2096.9274627811214
Ground: 18.74137809465192
Manmade: 57.46342281160086
Vegetation: 46.27912270652474


Weights sum to one: False (2331.3301055726024)
weights: [   1.11342454  110.80529464 2096.92746278   18.74137809   57.46342281
   46.27912271]


In [5]:
mu = sem_cloud_features[:,0,:,:].mean()
sigma = sem_cloud_features[:,0,:,:].std()
print(f'We have a mean of {mu} and a std of {sigma}')

We have a mean of 1.4801858478096677 and a std of 8.058974821123359
